In [132]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.optimize import minimize

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

# fundamental variables
simulate_theta = 3.14
beta = 0.95
N = 10000 #number of simulations
tolerance = 0.000001

# 🛠️ Work vs. Rest Fatigue Model (Deterministic)

In this model, an agent chooses each day between **working** and **resting**, balancing income with the long-term cost of fatigue.

---

### 📦 Model Setup

- **State**:  
  $x \in \{0, 1, 2, 3, 4\}$ — fatigue level  
  (0 = rested, 4 = exhausted)

- **Actions**:  
  $d = 0$ (rest), $d = 1$ (work)

---

### 💰 Utility Function

$$
u(x, d) = 
\begin{cases}
3 & \text{if } d = 1 \text{ and } x < 4 \\\\
- \infty & \text{if } d = 1 \text{ and } x = 4 \\\\
- \theta \cdot x & \text{if } d = 0
\end{cases}
$$

- $\theta$: fatigue cost (true value used in simulation = 1.2)

---

### 🔄 State Transitions

$$
x' =
\begin{cases}
\min(x + 1, 4) & \text{if } d = 1 \\\\
\max(x - 1, 0) & \text{if } d = 0
\end{cases}
$$

---

### 🎯 Objective

The agent maximizes:

$$
V(x) = \log \left( \sum_{d \in D(x)} \exp(u(x,d) + \beta V(x')) \right)
$$

- Choice probabilities follow logit structure
- Simulated over $T = 1000$ days
- Goal: **estimate** the fatigue cost parameter $\theta$

In [133]:
# BUILDING MODELS: defining functions

def next_state(x,d):
    if d == 1:
        return min(x+1,4)
    if d == 0:
        return max(x-1,0)
    
def utility(x,d,theta):
    if d == 1:
        if x == 4:
            return float("-inf")
        elif x < 4:
            return 3
    if d == 0:
        return -(theta*x)

def iteration(value_f, theta): #iterate in-place
    iterated = value_f.copy()
    for i in range(4): #iterate over all states
        state = i
        v0 = np.exp(utility(state,0, theta) + beta*iterated[next_state(state,0)])
        v1 = np.exp(utility(state,1, theta) + beta*iterated[next_state(state,1)])
        value_f[i] = np.log(v0 + v1)
    # only one decision on date 4    
    value_f[4] = np.log(np.exp(utility(4,0, theta) + beta*iterated[next_state(4,0)]))

def solve_value_f(value_f, theta, tolerance): #solving for value function
    distance = 1
    while distance > tolerance:
        temp = value_f.copy()
        iteration(value_f,theta)
        distance = sum(abs(value_f - temp))
    return value_f

def choice_value(value_f, x,d, theta):
    return utility(x,d, theta) + beta*value_f[next_state(x,d)]

def choice_prob_matrix(value_f, theta): #given a converged value function and theta
    matrix = [0]* 5
    for i in range(4):
        prob_1 = np.exp(choice_value(value_f,i,1,theta))/(np.exp(choice_value(value_f,i,1,theta)) + np.exp(choice_value(value_f,i,0,theta)))
        prob_0 = 1 - prob_1
        matrix[i] = [prob_0, prob_1]
    matrix[4] = [1,0]
    return matrix 


In [134]:
# SIMULATION
record = []
state = 0

# SOLVE firstly for the model with theta = 1.2 
V = np.zeros(5)
solve_value_f(V,simulate_theta,tolerance)
print("converged value function",V)
choice_prob = choice_prob_matrix(V, simulate_theta)
print("choice matrix", choice_prob)

# # simulation now 
for i in range(N):
    # print(probs)
    probs = [choice_prob[state][d] for d in [0,1]]
    action = np.random.choice([0, 1], p=probs)
    record.append((state, action))
    state = next_state(state,action)
    # another way to start next_state
    # state = np.random.choice([0, 1,2,3,4], p=[0.2,0.2,0.2,0.2,0.2])

# print(record)
#now we have a record for our theta of some value 

converged value function [ 10.008   6.396  -0.201  -9.611 -21.69 ]
choice matrix [[0.60628422466331056, 0.39371577533668944], [0.97232036455539461, 0.027679635444605378], [0.99733899018189165, 0.0026610098181083872], [0.99966283372127762, 0.00033716627872237159], [1, 0]]


In [135]:
# ESTIMATION:  using the record of (state, action)

def likelihood_function(theta):
    theta = theta[0]
    # firstly solve model 
    V = np.zeros(5)
    solve_value_f(V, theta, tolerance)
    choice_prob = choice_prob_matrix(V, theta)
    likelihood_sum = 0
    for i in range(N):
        likelihood_sum += np.log(max(choice_prob[record[i][0]][record[i][1]], 1e-10))
    return -likelihood_sum

result = minimize(likelihood_function, 0)
# print(result)
print("theta cost variable found to be equal", result.x[0])


theta cost variable found to be equal 3.14982981421
